In [1]:
import torch
from torch import nn, optim
from sklearn.datasets import load_boston
from torch.utils.data import  TensorDataset, DataLoader

In [2]:
boston = load_boston()

X_train = boston.data[:120]
y_train = boston.target[:120]

X_test = boston.data[120:]
y_test = boston.target[120:]

X_train, y_train, X_test, y_test = map(torch.tensor, 
                                       (X_train, y_train, X_test, y_test))

n, c = X_train.shape

print(X_train.shape)
print(y_train.min(), y_train.max())

torch.Size([120, 13])
tensor(12.7000, dtype=torch.float64) tensor(43.8000, dtype=torch.float64)


In [42]:
import math

weights = torch.randn(c, 1, dtype=torch.float64) / math.sqrt(c)
weights.requires_grad_()
bias = torch.zeros(1, requires_grad=True)

In [54]:
def relu(x):
    return torch.maximum(torch.zeros_like(x), x)

def model(xb):
    return relu(xb @ weights + bias)+20

In [55]:
batch_size = 5

xb = X_train[0:batch_size]
preds = model(xb)
print(preds[0], preds.shape)

tensor([20.], dtype=torch.float64, grad_fn=<SelectBackward0>) torch.Size([5, 1])


In [62]:
def mse(input_data, target):
    return torch.square(input_data - target).mean()

loss_func = mse

In [63]:
yb = y_train[0:batch_size]
print(loss_func(preds, yb))

tensor(135.3300, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [64]:
import math

weights = torch.randn(c, 1, dtype=torch.float64) / math.sqrt(c)
weights.requires_grad_()
bias = torch.zeros(1, requires_grad=True)

lr = 0.5
epochs = 2
batch_size = 5

for epoch in range(epochs):
    print(f'{epoch}: {loss_func(model(X_test), y_test)}')
    for i in range((n-1)//batch_size+1):
        start_i = i * batch_size
        end_i = start_i + batch_size
        xb = X_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        
        pred = model(xb)
        
        loss = loss_func(pred, yb)
        
        loss.backward()
        
        with torch.no_grad():
            weights -= weights.grad *lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()

0: 534.5127057233656
1: 108.35880829015544


In [134]:
import torch.nn.functional as F

def model(xb):
    return F.relu(xb @ weights + bias)

In [137]:
print(loss_func(model(xb), yb))

tensor(388.3240, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [141]:
from torch import nn

class myModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(
            torch.randn(c, 1, dtype=torch.float64) / math.sqrt(c))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, xb):
        return F.relu(xb @ self.weights + self.bias)

In [142]:
model = myModel()
print(loss_func(model(xb), yb))

tensor(388.3240, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [146]:
def fit():
    for epoch in range(epochs):
        for i in range((n - 1) // batch_size + 1):
            start_i = i * batch_size
            end_i = start_i + batch_size
            xb = X_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()

fit()

In [3]:
class myModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(c, 1)

    def forward(self, xb):
        return self.lin(xb)

In [4]:
model = myModel()
print(loss_func(model(xb), yb))

NameError: name 'loss_func' is not defined

In [73]:
from torch import optim
import torch.nn.functional as F

loss_func = F.mse_loss

lr = 0.05
epochs = 200
batch_size = 5

n, c = X_train.shape

print(c)

class myModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(c, 1, dtype=torch.float64)
        self.relu = F.relu

    def forward(self, xb):
        return self.relu(self.lin(xb))

def get_model():
    model = myModel()
    return model, optim.SGD(model.parameters(), lr=lr)

model, opt = get_model()
# print(loss_func(model(xb), yb))

for epoch in range(epochs):
    for i in range((n - 1) // batch_size + 1):
        start_i = i * batch_size
        end_i = start_i + batch_size
        xb = X_train[start_i:end_i]
        yb = y_train[start_i:end_i].unsqueeze(-1)
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        opt.zero_grad()
        loss.backward()
        opt.step()
        
    if epoch % 10 == 0:
        print(f'{epoch}/{epochs}:')
        print(loss_func(model(xb), yb))

13
0/200:
tensor(388.3240, dtype=torch.float64, grad_fn=<MseLossBackward0>)
10/200:
tensor(388.3240, dtype=torch.float64, grad_fn=<MseLossBackward0>)
20/200:
tensor(388.3240, dtype=torch.float64, grad_fn=<MseLossBackward0>)
30/200:
tensor(388.3240, dtype=torch.float64, grad_fn=<MseLossBackward0>)
40/200:
tensor(388.3240, dtype=torch.float64, grad_fn=<MseLossBackward0>)
50/200:
tensor(388.3240, dtype=torch.float64, grad_fn=<MseLossBackward0>)
60/200:
tensor(388.3240, dtype=torch.float64, grad_fn=<MseLossBackward0>)
70/200:
tensor(388.3240, dtype=torch.float64, grad_fn=<MseLossBackward0>)
80/200:
tensor(388.3240, dtype=torch.float64, grad_fn=<MseLossBackward0>)
90/200:
tensor(388.3240, dtype=torch.float64, grad_fn=<MseLossBackward0>)
100/200:
tensor(388.3240, dtype=torch.float64, grad_fn=<MseLossBackward0>)
110/200:
tensor(388.3240, dtype=torch.float64, grad_fn=<MseLossBackward0>)
120/200:
tensor(388.3240, dtype=torch.float64, grad_fn=<MseLossBackward0>)
130/200:
tensor(388.3240, dtype=t